# 4_1_08 — Congruencia entre ingresos netos por representación legal de la pareja/dependientes y remuneración declarada
| Escenario                                                         | Condición                                                            | Resultado        |
| ----------------------------------------------------------------- | -------------------------------------------------------------------- | ---------------- |
| No aplica                                                         | Tipo de representación **NO es** “Pareja” ni “Dependiente económico” | ⚪ **NO_APLICA**  |
| Tiene representación (pareja/dep.) pero no reporta monto          | Monto mensual neto `<= 0` o nulo                                     | ⚪ **SIN_DATO**   |
| Tiene representación con monto > 0, pero el ingreso B es 0 o nulo | B `<= 0`                                                             | 🔴 **NO_CUMPLE** |
| Tiene representación con monto > 0 **y B > 0**                    | B `> 0`                                                              | 🟢 **CUMPLE**    |
| Información insuficiente                                          | Falta tipo, monto o ingreso B                                        | ⚪ **SIN_DATO**   |


In [ ]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "4_1_08_CONGRUENCIA_REPRESENTACION_PAREJA_INGRESOS"

# --- Auxiliares ---

def extraer_valor(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None
        return actual if actual not in ("", " ") else None
    except:
        return None

def normalizar_texto(v):
    if v is None:
        return None
    return str(v).strip().upper()

def extraer_float(v):
    try:
        return float(v)
    except:
        return None


# --- Evaluación de la métrica ---

def evaluar_metrica(doc):

    # Representación legal (pareja/dependiente)
    tipo_repr = normalizar_texto(
        extraer_valor(doc, "declaracion.intereses.representacion.tipo")
    )

    monto_repr = extraer_float(
        extraer_valor(doc, "declaracion.intereses.representacion.montoMensual")
    )

    # Ingreso anual neto de la pareja/dependientes – apartado B
    ingreso_b = extraer_float(
        extraer_valor(doc,
            "declaracion.situacionPatrimonial.ingresos.totalIngresosMensualesNetos"
        )
    )

    # --- Lógica institucional ---

    # 1. Solo aplica si tipo = PAREJA o DEPENDIENTE
    if tipo_repr not in ("PAREJA", "DEPENDIENTE ECONÓMICO", "DEPENDIENTE ECONOMICO"):
        return "NO_APLICA"

    # 2. Si tiene representación pero no monto → SIN_DATO
    if monto_repr is None or monto_repr <= 0:
        return "SIN_DATO"

    # 3. Tiene monto > 0 → validar ingresos B
    if ingreso_b is None:
        return "SIN_DATO"

    if ingreso_b > 0:
        return "CUMPLE"

    return "NO_CUMPLE"


# --- Procesamiento MongoDB ---

def procesar_metrica_4_1_08():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.intereses.representacion": 1,
            "declaracion.situacionPatrimonial.ingresos": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error en doc {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne({"_id": doc["_id"]},
                          {"$set": {METRIC_ID: resultado}},
                          upsert=True)
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f"> Procesados {total} documentos...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}")
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass
        print("\nConexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_1_08()


Procesando métrica 4_1_08_CONGRUENCIA_REPRESENTACION_PAREJA_INGRESOS...



c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


> Procesados 2000 documentos...
> Procesados 4000 documentos...
> Procesados 6000 documentos...
> Procesados 8000 documentos...
> Procesados 10000 documentos...
> Procesados 12000 documentos...
> Procesados 14000 documentos...
> Procesados 16000 documentos...
> Procesados 18000 documentos...
